# Repeat from 4e1

In [1]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd

from sklearn.preprocessing import QuantileTransformer
from joblib import load

In [2]:
pipe = load("mypipe.joblib")

In [3]:
server = 'fs.rice.edu'
database = 'stocks'
username = 'stocks'
password = '6LAZH1'
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database 

conn = create_engine(string).connect()

In [4]:
df = pd.read_sql(
    """
    select ticker, bm, mom12m, roeq, siccd
    from ghz
    order by ticker
    """,
    conn
)
df = df.dropna()
df = df.set_index("ticker")
features = ["bm", "mom12m", "roeq"]

In [5]:
qt = QuantileTransformer(output_distribution="normal")
df[features] = qt.fit_transform(df[features])

In [6]:
inds = pd.read_csv("siccodes12.csv", index_col="industry")
ind_names = inds.index.unique().to_list()

def industry(sic):
  try:
    return inds[(inds.start<=sic)&(sic<=inds.end)].index[0]
  except:
    return "Other"
    
df["industry"] = df.siccd.map(industry)
features.append("industry")

In [7]:
df["predict"] = pipe.predict(df[features])

# Additional imports

In [9]:
from datetime import datetime
from pathlib import Path
import numpy as np
import time

from alpaca.trading.client import TradingClient
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce

KEY = "PK11EGE7U1KLVJDCOUKH"
SECRET_KEY = "oGTDIwcIgRajhyM0yq1knAjzjLtfQQ6xUTA4uBfp" 

data_client = StockHistoricalDataClient(KEY, SECRET_KEY)
trading_client = TradingClient(KEY, SECRET_KEY, paper=True)

# Tradable and shortable

In [10]:
assets = trading_client.get_all_assets()

tradable = {x.symbol:x.tradable for x in assets}
df["tradable"] = [tradable[tick] if tick in tradable else False for tick in df.index]

shortable = {x.symbol:x.shortable for x in assets}
df["shortable"] = [shortable[tick] if tick in shortable else False for tick in df.index]

# Equity and current positions

In [11]:
account = trading_client.get_account()
equity = float(account.equity)

current = trading_client.get_all_positions()
current = {x.symbol: int(x.qty) for x in current}
df["current"] = [current[tick] if tick in current else 0 for tick in df.index]


# Ranking tradable and shortable stocks

In [12]:
df["predict_long"] = np.where(df.tradable, df.predict, np.nan)
df["rank_from_top"] = df.predict_long.rank(
  ascending=False, 
  method="first"
)

df["predict_short"] = np.where(df.shortable, df.predict, np.nan)
df["rank_from_bottom"] = df.predict_short.rank(method="first")

# Longs and shorts

In [13]:
numlong = numshort = 100

df["long"] = df.rank_from_top <= numlong
df["short"] = df.rank_from_bottom <= numshort

print(f"number long is {df[df.long].shape[0]}")
print(f"number short is {df[df.short].shape[0]}")

df_all = df.copy()
df = df[df.long | df.short | (df.current !=0)].copy()

number long is 100
number short is 100


# Today's prices

In [16]:
bars

{}

In [14]:
today = datetime.today().date()
y, m, d = today.year, today.month, today.day

request_params = StockBarsRequest(
                    symbol_or_symbols=df.index.to_list(),
                    timeframe=TimeFrame.Day,
                    start=datetime(y, m, d)
                    )

bars = data_client.get_stock_bars(request_params)

df["price"] = bars.df["close"].reset_index(level=1, drop=True)
df = df[~df.price.isna() & (df.price!=0)]

KeyError: "None of ['symbol', 'timestamp'] are in the columns"

# Targets and trades

In [ ]:
df["target_dollars"] = (
  (1.3*equity/numlong)*df.long - 
      (0.3*equity/numshort)*df.short
)
df["current_dollars"] = df.current*df.price
df["trade_dollars"] = df.target_dollars - df.current_dollars
df["trade"] = (df.trade_dollars / df.price).astype(int)

# for going from long to short or vice versa
def two_trades(row): 
    if ((np.sign(row.current) != np.sign(row.trade)) and (row.current!=0) and  (abs(row.trade) > abs(row.current))):
        return True 
    return False 
df["two_trades"] = df.apply(two_trades, axis=1)

print(f"number of tickers with two trades is {df[df.two_trades].shape[0]}")

number of tickers with two trades is 0


# Submit trades

In [15]:
for tick in df[df.trade!=0].index:
  market_order_data = MarketOrderRequest(
    symbol=tick,
    qty=abs(df.current[tick]) if df.two_trades[tick] else abs(df.trade[tick]),
    side=OrderSide.BUY if df.trade[tick]>0 else OrderSide.SELL,
    time_in_force=TimeInForce.GTC
  )
  market_order = trading_client.submit_order(
    order_data=market_order_data
  )

# execute second trades if any
if df[df.two_trades].shape[0] > 0:
  time.sleep(5)
  for tick in df[df.two_trades].index:        
    market_order_data = MarketOrderRequest(
      symbol=tick,
      qty=abs(df.trade[tick]) - abs(df.current[tick]),
      side=OrderSide.BUY if df.trade[tick]>0 else OrderSide.SELL,
      time_in_force=TimeInForce.GTC
    )
    market_order = trading_client.submit_order(
      order_data=market_order_data
    )


# Save data

In [16]:
df = df_all.join(
    df[["trade_dollars", "current_dollars", "trade", "two_trades"]],
    how="left" 
)

today = datetime.today().strftime("%Y-%m-%d")
df["date"] = today
df = df[["date", "trade"]].reset_index()

df["date"] = today

if Path('./trades.csv').is_file():
    d = pd.read_csv("trades.csv")
    d = d[d.date != today]
    d = pd.concat((d, df))
    df.to_csv("trades.csv")
else:
    df.to_csv("trades.csv")

account = trading_client.get_account()
account = dict(account)
account = pd.DataFrame(pd.Series(account)).T
account["date"] = today

if Path('./account.csv').is_file():
    d = pd.read_csv("account.csv")
    d = d[d.date != today]
    d = pd.concat((d, account))
    d.to_csv("account.csv")
else:
    account.to_csv("account.csv")

positions = trading_client.get_all_positions()
positions = {x.symbol: x.qty for x in positions}
positions = pd.DataFrame(pd.Series(positions))
positions["date"] = today

if Path('./positions.csv').is_file():
    d = pd.read_csv("positions.csv")
    d = d[d.date != today]
    d = pd.concat((d, positions))
    d.to_csv("positions.csv")
else:
    positions.to_csv("positions.csv")


C:\Users\keb7\AppData\Local\Temp\ipykernel_57056\3783317904.py:28: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  d = pd.concat((d, account))
